In [8]:
from importlib import reload
import os
from src import utils
import re
import pandas as pd
reload(utils)

data_in_path  = 'Database pdfs/Total/word-extras'
data_out_path = 'Database pdfs/Total/word-extras-noduplicates'

In [9]:
!pip install pandas


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
all_files_titles=[]
no_titles_files=[]

In [11]:
re.find

AttributeError: module 're' has no attribute 'find'

In [12]:
def clean_title(title_text):
    if title_text is None or len(title_text)==0:
        raise Exception("empty title")

    regex_list=["\(Lab Tests and Diagnostic Procedures\)","\(Lexi.*?\)"]
    regex="|".join([f"({reg})" for reg in regex_list])
    result=re.search(regex,title_text)
    if result is None:
        result=re.search("\(.+\)$",title_text)
    if result is not None:
        sub_title=result.group()
        title=title_text[:result.start()]
    else:
        raise Exception("title pattern not found")
    title=title.strip()

    return  title,sub_title

# check  file titles

In [16]:
file_names=os.listdir(data_in_path)
corrupted=[]
for file_name in file_names:
    document=utils.read_doc(f"{data_in_path}/{file_name}")
    title=utils.get_title(document)
    try:
        clean_title(title)
    except Exception as e:
        print(title)
        print(e)
        print(file_name)
        corrupted.append(file_name)


empty title
1324Lexicomp.docx


In [17]:
len(corrupted)

1

In [121]:
p="C:\\projects\\drugDatabase\\wordv2"

for file in corrupted:
    os.system(f"{p}\\{file}")

- i found that all files without titles have no data so i removed them

In [21]:
with open("Database pdfs/Total/metadata/deleted.txt",'a') as f:
    for file in corrupted:
        f.write(file)
        f.write("\n")
        os.remove(f"{data_in_path}/{file}")

__________________________

In [22]:
file_names=os.listdir(data_in_path)
data_list=[]
for file_name in file_names:
    document=utils.read_doc(f"{data_in_path}/{file_name}")
    title,sub_title=clean_title(utils.get_title(document))
    words_count=utils.words_count(document)
    data_list.append([file_name,title,sub_title,words_count])

In [23]:
titles_df=pd.DataFrame(data_list,columns=['file_name','title','sub_title','words_count'])

In [24]:
titles_df.to_csv("./Database pdfs/Total/metadata/drug_files.csv")

In [25]:
titles_duplicated=titles_df[titles_df.duplicated(subset=['title'])]

In [26]:
len(titles_duplicated)

1113

In [27]:
titles_count_duplicated=titles_df[titles_df.duplicated(subset=['title','words_count'])]

In [29]:
len(titles_count_duplicated)

969

In [30]:
len(titles_df)

2140

In [31]:
duplicated_titles=titles_count_duplicated.sample(5)['title']

In [32]:
titles_df[titles_df.title.isin(duplicated_titles)].sort_values(by='title')

,file_name,title,sub_title,words_count
1331,718Lexicomp.docx,Ampicillin and Cloxacillin [INT],(Lexi-Drugs International [Concise]),224
176,116Lexicomp.docx,Ampicillin and Cloxacillin [INT],(Lexi-Drugs International [Concise]),224
1106,504Lexicomp.docx,Ampicillin and Cloxacillin [INT],(Lexi-Drugs International [Concise]),224
1180,574Lexicomp.docx,Ampicillin and Cloxacillin [INT],(Lexi-Drugs International [Concise]),224
1420,802Lexicomp.docx,Doxycycline Hyclate Periodontal Extended-Relea...,(Lexi-Drugs),2125
1369,755Lexicomp.docx,Doxycycline Hyclate Periodontal Extended-Relea...,(Lexi-Drugs),2125
251,1241Lexicomp.docx,EPHEDrine (Systemic),(Lexi-Drugs),4048
303,128Lexicomp.docx,EPHEDrine (Systemic),(Lexi-Drugs),4048
341,1325Lexicomp.docx,EPHEDrine (Systemic),(Lexi-Drugs),4048
844,257Lexicomp.docx,EPHEDrine (Systemic),(Lexi-Drugs),4048


In [33]:
titles_df.to_excel('Titles with duplication.xlsx')

In [234]:
p="C:\\projects\\drugDatabase\\wordv2"

file="164Lexicomp.docx"
document=utils.read_doc(f"{p}\\{file}")
# print(utils.words_count(document))
os.system(f"{p}\\{file}")


file="420Lexicomp.docx"
os.system(f"{p}\\{file}")
document=utils.read_doc(f"{p}\\{file}")
# utils.words_count(document)

In [176]:
file="1Lexicomp.docx"
document=utils.read_doc(f"{p}\\{file}")
utils.words_count(document)

7733

In [230]:
no_duplicates_df=titles_df.drop_duplicates(subset=['title','words_count'])

In [236]:
no_duplicates_df.to_csv("../metadata/drug_no_duplicates.csv")

In [233]:
duplicated_titles = no_duplicates_df[no_duplicates_df.duplicated('title')].sample(5)['title']
no_duplicates_df[no_duplicates_df.title.isin(duplicated_titles)].sort_values(by='title')


,file_name,title,sub_title,words_count
673,164Lexicomp.docx,Amphotericin B Cholesteryl Sulfate Complex,(Lexi-Drugs),4610
1017,420Lexicomp.docx,Amphotericin B Cholesteryl Sulfate Complex,(Lexi-Drugs),4608
473,1455Lexicomp.docx,Ciprofloxacin (Systemic),(Lexi-Drugs),23994
1638,Ciprofl oxacin (Systemic).docx,Ciprofloxacin (Systemic),(Lexi-Drugs),23999
1642,Ciprofloxacin (Systemic) (Lexi-Drugs).docx,Ciprofloxacin (Systemic),(Lexi-Drugs),24004
826,241Lexicomp.docx,DilTIAZem,(Lexi-Drugs),19096
1624,999Lexicomp.docx,DilTIAZem,(Lexi-Drugs),19098
273,1261Lexicomp.docx,Liothyronine,(Lexi-Drugs),6679
940,34Lexicomp.docx,Liothyronine,(Lexi-Drugs),6654
134,1130Lexicomp.docx,Terbutaline,(Lexi-Drugs),6948


In [232]:

no_duplicates_df[no_duplicates_df.duplicated('title')].sort_values('title')

,file_name,title,sub_title,words_count
139,1135Lexicomp.docx,Ado-Trastuzumab Emtansine,(Lexi-Drugs),5513
1592,966Lexicomp.docx,Aminophylline,(Lexi-Drugs),6957
1017,420Lexicomp.docx,Amphotericin B Cholesteryl Sulfate Complex,(Lexi-Drugs),4608
902,314Lexicomp.docx,Bivalirudin,(Lexi-Drugs),6898
333,1317Lexicomp.docx,C1 Inhibitor (Human),(Lexi-Drugs),5105
329,1313Lexicomp.docx,CARBOplatin,(Lexi-Drugs),20033
654,1632Lexicomp.docx,Ceftazidime and Avibactam,(Lexi-Drugs),5523
1589,963Lexicomp.docx,Chloramphenicol (Systemic),(Lexi-Drugs),4993
1642,Ciprofloxacin (Systemic) (Lexi-Drugs).docx,Ciprofloxacin (Systemic),(Lexi-Drugs),24004
1638,Ciprofl oxacin (Systemic).docx,Ciprofloxacin (Systemic),(Lexi-Drugs),23999


In [240]:
titles_df[titles_df.title.str.find("Epoetin Alfa")!=-1]

,file_name,title,sub_title,words_count
0,0Lexicomp.docx,Epoetin Alfa,(Lexi-Drugs),10559
566,1543Lexicomp.docx,Epoetin Alfa,(Lexi-Drugs),10580
923,333Lexicomp.docx,Epoetin Alfa,(Lexi-Drugs),10559
1031,434Lexicomp.docx,Epoetin Alfa,(Lexi-Drugs),10559
1095,496Lexicomp.docx,Epoetin Alfa,(Lexi-Drugs),10559
1187,582Lexicomp.docx,Epoetin Alfa,(Lexi-Drugs),10559


In [241]:
eoptien=titles_df[titles_df.title.str.find("Epoetin Alfa")!=-1]

for i in range(len(eoptien)):

    os.system(f"{p}\\{eoptien.iloc[i]['file_name']}")